# 勾配ブースティング

弱い学習器をいっぱい集めたらマシな学習器になるでしょうか？
歴史的には[Kearns](https://www.cis.upenn.edu/~mkearns/papers/crypto.pdf)がこの疑問に取り組み始め，Adaboostと呼ばれるBoostingアルゴリズムにつながっていきました．今回は勾配ブースティングについて見ていきます．

特に次のPAC（Probability Approximately Correct）学習と弱学習器について考えます：

---

**PAC学習**

ある真の仮説$h^*$がPAC学習可能であるとは，任意の近似誤差$\epsilon$と$\delta$に対して，以下を満たす仮説$h$を出力する多項式時間アルゴリズムが存在すること

$$
\mathbb{P}\left[\mathbb{P}\left[h \neq h^*(x)\right]>\epsilon\right] \leq 1-\delta
$$

---

**弱学習器**

固定された$\epsilon$と$\delta$で学習可能な学習器のこと．

---


このとき，弱学習器から強学習器（PAC学習を実現する学習器）は作れるか？について取り組んでいきます．
基本的には次の形式の学習器について考えます：

**基本的な集団学習アルゴリズム**

* 観測データ：$\mathcal{D}=\left\{y_i, x_i\right\}_{i=1}^n\left(y_i=\{-1,+1\}, \boldsymbol{x}_i \in \mathcal{X} \subset \mathbb{R}^d\right)$
* $T$をアンサンブルする数とします．ステップ$t=1, 2, \dots, T$に対して，
  * $\mathcal{D}$からデータ$\mathcal{D}_t$を生成
  * $\mathcal{D}_t$から識別器$h_t$を求める
* $\left\{h_t: \mathcal{X} \rightarrow\{-1,+1\}\right\}_{t=1}^T$を組み合わせて識別器$H$を構築

します．特に$\mathcal{D}_t$を$\mathcal{D}$からサンプリングして作成するアルゴリズムをバギング，$\mathcal{D}_t$を$\mathcal{D}$と$\{h_1, \dots, h_{t-1}\}$の出力を利用して作成するアルゴリズムをブースティングといいます．

---


## 回帰におけるGradient boosting

さて，回帰を実現するGradient boostingアルゴリズムについて見ていきましょう．先に回帰について確認して，その仕組み（なぜ勾配なのか？）を見ます．

* 観測データ：$\left\{\left(y_i, \boldsymbol{x}_i\right)\right\}_{i=1}^n \boldsymbol{x}_i \in \mathcal{X} \subset \mathbb{R}^d, y_i \in \mathbb{R}$
* 弱学習器：$h: \mathcal{X} \rightarrow \mathbb{R}$
* 集団学習における回帰問題($H(x_i)$は弱学習器を混ぜることで実現します)：
$$
y_i=H\left(\boldsymbol{x}_i\right)=\sum_{t=1}^T \alpha_t h_t\left(\boldsymbol{x}_i\right) .(i=1,2, \ldots, n)
$$

以降，弱学習器$h_t$の選び方と，その混ぜ方$\alpha_t$について考えていきます．

---

**モチベーション**

まずはモチベーションを確認しましょう．結局，弱学習器を追加するときに，各$(y_i, x_i)$に対して
$$
y_i \approx H_{t-1}\left(\boldsymbol{x}_i\right)+\alpha_t h_t\left(\boldsymbol{x}_i\right)(i=1,2, \ldots, n)
$$
を満たすように$h_t$を選びたいので，

$$
y_i-H_{t-1}\left(\boldsymbol{x}_i\right) \approx \alpha_t h_t\left(\boldsymbol{x}_i\right)(i=1,2, \ldots, n)
$$

が成立すればOKです．よって，

$$
\left\{\left(\tilde{y}_i, \boldsymbol{x}_i\right)\right\}_{i=1}^n,\left(\tilde{y}_i=y_i-H_{t-1}\left(\boldsymbol{x}_i\right)\right)
$$

を使って弱学習器$h_t$を学べば良さそうです．すなわち，これまで構成した学習器$H_{t-1}$の残差$\tilde{y}_i$を使って学習器を構成しているわけですね．

これまでの学習器$H_{t-1}$の損失関数を，回帰なので二乗損失として，
$$
\mathcal{L}\left(H_{t-1}\right)=\sum_{i=1}^n \ell\left(y_i, H_{t-1}\left(\boldsymbol{x}_i\right)\right)=\frac{1}{2} \sum_{i=1}^n\left(y_i-H_{t-1}\left(\boldsymbol{x}_i\right)\right)^2
$$
とします．$H_{t-1}(x_i)$を一つの変数として偏微分を取ると，
$$
\frac{\partial \mathcal{L}\left(H_{t-1}\right)}{\partial H_{t-1}\left(\boldsymbol{x}_i\right)}=H_{t-1}\left(\boldsymbol{x}_i\right)-y_i .
$$
なので，残差が出てきました！よって，残差を実現する$h_t$は
$$
h_t\left(\boldsymbol{x}_i\right)=y_i-H_{t-1}\left(\boldsymbol{x}_i\right)=-\frac{\partial L}{\partial H_{t-1}\left(\boldsymbol{x}_i\right)}
$$
を満たし，その場合は
$$
\begin{aligned}
H_t\left(\boldsymbol{x}_i\right) & =H_{t-1}\left(\boldsymbol{x}_i\right)+\alpha_t h_t\left(\boldsymbol{x}_i\right) \\
& =H_{t-1}\left(\boldsymbol{x}_i\right)-\alpha_t \frac{\partial L}{\partial H_{t-1}\left(\boldsymbol{x}_i\right)}
\end{aligned}
$$
が成り立ちます．これは勾配法そのものですね．$h_t$は直接残差を取れるわけではないので，$h_t$を回帰しましょう：

$$
\begin{aligned}
\tilde{y}_i & =-\left[\frac{\partial \ell\left(y_i, H\left(\boldsymbol{x}_i\right)\right)}{\partial H\left(\boldsymbol{x}_i\right)}\right]_{H=H_{t-1}}=y_i-H_{t-1}\left(\boldsymbol{x}_i\right) \\
h_t & =\underset{h \in \mathcal{H}}{\operatorname{argmin}} \frac{1}{2} \sum_{i=1}^n\left(\tilde{y}_i-h\left(\boldsymbol{x}_i\right)\right)^2
\end{aligned}
$$

とすれば，結局$h_t$の学習は，「学習データ全体についての勾配」を求めていることになります．

まとめると，Gradient boostingは次のアルゴリズムを走らせます：

* ステップ$t=1, 2, \dots, T$に対して：
  * $\ell(y, H(\boldsymbol{x}))$の関数$H$に対する勾配を計算する：
$$
\tilde{y}_i=-\left[\frac{\partial \ell\left(y_i, H\left(\boldsymbol{x}_i\right)\right)}{\partial H\left(\boldsymbol{x}_i\right)}\right]_{H=H_{t-1}}
$$
  * 勾配を回帰：$h_t=\underset{h \in \mathcal{H}}{\operatorname{argmin}} \sum^n\left(\tilde{y}_i-h\left(\boldsymbol{x}_i\right)\right)^2$
  * 学習率を調整：$\alpha_t=\underset{\alpha>0}{\operatorname{argmin}} \mathcal{L}\left(H_{t-1}+\alpha h_t\right)$
  * 次の学習器を生成：$H_t=H_{t-1}+\alpha_t h_t$

最後に
$$
H(\boldsymbol{x})=\sum_{t=1}^T \alpha_t h_t(\boldsymbol{x})
$$
を出力します．



---

**損失関数の例**

* 二乗損失：$\ell\left(y_i, H\left(\boldsymbol{x}_i\right)\right)=\frac{1}{2}\left(y_i-H\left(\boldsymbol{x}_i\right)\right)^2, \quad \tilde{y}_i=y_i-H_{t-1}\left(\boldsymbol{x}_i\right)$
* 絶対損失：$\ell\left(y_i, H\left(\boldsymbol{x}_i\right)\right)=\left|y_i-H\left(\boldsymbol{x}_i\right)\right|, \quad \tilde{y}_i=\operatorname{sign}\left(y_i-H_{t-1}\left(\boldsymbol{x}_i\right)\right)$
* ロジスティック損失（二値分類）：$\ell\left(y_i, H\left(\boldsymbol{x}_i\right)\right) =\log \left(1+\exp \left(-y_i H\left(\boldsymbol{x}_i\right)\right)\right)$として，$\tilde{y}_i =\frac{y_i}{1+\exp \left(y_i H_{t-1}\left(\boldsymbol{x}_i\right)\right)}$

また，多クラス分類の場合はカテゴリ$k = 1, \dots, K$について
$$H_k(\boldsymbol{x})=\sum_{t=1}^T \alpha_{t, k} h_{t, k}(\boldsymbol{x})$$
を構築し，損失関数を
$$
P_k(\boldsymbol{x})=\frac{\exp \left(H_k(\boldsymbol{x})\right)}{\sum_{k^{\prime}=1}^K \exp \left(H_{k^{\prime}}(\boldsymbol{x})\right)}
$$
とします．つまり各クラス$k$に対して二値分類器を用意しているわけですね．マージン関数とは異なり，ソフトマックス損失を使っていることに注意しましょう．

## Gradient Boostingの汎化誤差解析（二値分類）

ここまでは回帰についてみてきました．
以下の汎化誤差解析では基本的に二値分類を考えます（回帰はリプシッツ連続じゃなかったりしてめんどくさいです）．

次の凸包を考えます：

$$
\operatorname{conv}(\mathcal{F})=\left\{\sum_{k=1}^K \alpha_k f_k \mid K \in \mathbb{N}, \sum_{k=1}^K \alpha_k=1, \alpha_k \geq 0, f_k \in \mathcal{F}\right\}
$$
以降では，凸包のラデマッハ複雑度が$R_n(\operatorname{conv}(\mathcal{F}))=R_n(\mathcal{F})$を満たすことを使います．
ここで，
$$
R_n(\operatorname{conv}(\mathcal{F}))=\mathbb{E}[\sup _{F \in \operatorname{conv}(\mathcal{F})} \frac{1}{n} \sum_{i=1}^n \sigma_i \underbrace{\sum_{k=1}^K \alpha_k f_k\left(x_i\right)}_{=F\left(x_i\right)}] .
$$
としました．$F(x_i)$は，与えられた$x_i$に対して凸包によって与えられる関数の値です．

**証明**

$$
\mathcal{A}=\left\{\left(\alpha_1, \ldots, \alpha_K\right) \mid K \in \mathbb{N}, \alpha_k \geq 0, \frac{1}{K} \sum_{k=1}^K \alpha_k=1\right\}
$$
について，

$$
\begin{aligned}
& \sup _{F \in \operatorname{conv}(\mathcal{F})} \frac{1}{n} \sum_{i=1}^n \sigma_i \underbrace{\sum_{k=1}^K \alpha_k f_k\left(x_i\right)}_{=F\left(x_i\right)} \\
& =\sup _{\left(\alpha_1 \ldots, \alpha_K\right) \in \mathcal{A}} \sup _{f_1, \ldots, f_K \in \mathcal{F}} \frac{1}{n} \sum_{i=1}^n \sigma_i \sum_{k=1}^K \alpha_k f_k\left(x_i\right)
\end{aligned}
$$

が成り立ちます．ここで$K$は固定してるわけではなく，適当な$K$を持ってきています（最終的に$K$に依存しないバウンドが出てくるので大丈夫です）．
これに対して，
$$
\begin{aligned}
& \sup _{f_1, \ldots, f_K \in \mathcal{F}} \sup _{\left(\alpha_1, \ldots, \alpha_K\right) \in \mathcal{A}} \frac{1}{n} \sum_{i=1}^n \sigma_i \sum_{k=1}^K \alpha_k f_k\left(x_i\right) \\
= & \sup _{f_1, \ldots . f_K \in \mathcal{F}} \sup _{\left(\alpha_1 \ldots \ldots \alpha_K\right) \in \mathcal{A}} \frac{1}{n} \sum_{k=1}^K \alpha_k \sum_{i=1}^n \sigma_i f_k\left(x_i\right) \\
= & \sup _{f_1, \ldots, f_K \in \mathcal{F}} \max _k \frac{1}{n} \sum_{i=1}^n \sigma_i f_k\left(x_i\right) \\
= & \sup _{f \in \mathcal{F}} \frac{1}{n} \sum_{i=1}^n \sigma_i f\left(x_i\right)
\end{aligned}
$$
が成り立ちます．
３行目では$\alpha_k$の和が１なので，$\alpha_k$のsupを取るのは一番値がでかい$k$に対して$1$を割り当てるのと同じなことを使ってます

---

さて，弱学習器の集合を$\mathcal{F}$としましょう．そして，Boostingの仮説関数を$H(\boldsymbol{x})=\sum_{t=1}^T \alpha_t h_t(\boldsymbol{x})$とします（ここで$h_t \in \mathcal{F}$）です．
Boostingの仮説関数の集合を$\mathcal{H}$とします．

ここで，Boostingアルゴリズムでは$\alpha_t \geq 0$かつ$\sum_{t=1}^T \alpha_t=1$を考えていました．
これを実現するために，適当に持ってきた$\alpha_t$や$h_t$に対して，
* $\alpha_t$の値については，任意の$h \in \mathcal{F}$に対して$-h$も$\mathcal{F}$とする
* $\sum^t_{t=1} \alpha_t$を使って$H(x)$を正規化しても符号が変化しないので，正規化する（これは二値分類ではOKだが多値分類だと問題がでないか？）

ことにします．これで$\mathcal{F}$から適切な仮説が作れます．さて，次を満たす仮説集合を考えましょう．
$$
H(\boldsymbol{x})=\sum_{t=1}^T \alpha h_t(\boldsymbol{x}), h_t \in \mathcal{F}, \alpha_t \geq 0, \sum_{t=1}^t \alpha_t=1
$$
このとき次が成立します：

---

正数$\rho > 0$に対して，確率$1-\delta$以上で
$$
\begin{aligned}
\mathbb{P}[\operatorname{sign}(H(\boldsymbol{x})) \neq y] \leq & \frac{1}{n} \sum_{i=1}^n \phi_\rho\left(y_i H\left(\boldsymbol{x}_i\right)\right) \\
& +\frac{2}{\rho} R_n(\mathcal{F})+\sqrt{\frac{\log (1 / \delta)}{2 n}} .
\end{aligned}
$$
が成立する．

ここで損失関数を
$$
\phi_{\rho=1}\left(y_i H\left(\boldsymbol{x}_i\right)\right) \leq \exp \left(-y_i H\left(\boldsymbol{x}_i\right)\right)
$$
とすれば，Adaboostの汎化誤差解析になります（Gradient boostingは損失関数が決まってるわけではなく，特に指数損失の場合をAdaboostと呼ぶらしいです．指数損失のほうがヒンジ損失よりでかいので，Adaboostはあまり誤差を許容しません．なのでラベルミスなどにとても弱いです）．

**証明**

これはそんなに難しくありません．
簡単な計算で，元の

$$
\begin{aligned}
\mathbb{P}[\operatorname{sign}(H(\boldsymbol{x})) \neq y]= & \mathbb{E}[\mathbb{I}(\operatorname{sign}(H(\boldsymbol{x})) \neq y)] \\
= & \mathbb{E}[\mathbb{I}(y H(\boldsymbol{x}) \leq 0)] \\
\leq & \mathbb{E}\left[\phi_\rho(y H(\boldsymbol{x}))\right] \\
\leq & \frac{1}{n} \sum_{n=1}^n \phi_\rho\left(y_i H\left(\boldsymbol{x}_i\right)\right) \\
& +2 \underbrace{R_n\left(\phi_\rho \circ \mathcal{H}\right)}_{=\frac{1}{\rho} R_n(\mathcal{F})}+\sqrt{\frac{\log (1 / \delta)}{2 n}} .
\end{aligned}
$$
が成り立ちます．


## 決定株（decision stamp）のラデマッハ複雑度

他の重要な集団学習アルゴリズムに決定木アルゴリズムがあります．
決定木は次の決定株アルゴリズムの集団学習とみなせます．先に決定株について見てみましょう：

* 入力：$x \in \mathcal{X} \subset \mathbb{R}^d$
* 決定株のパラメータ：$\alpha \in\{-1,+1\}, k \in\{1, \ldots, d\}, z \in \mathbb{R}$
* 仮説関数：$h_{\alpha, k, z}(x)=\alpha \cdot \operatorname{sign}\left(x_k-z\right)$
    * すなわち，入力$x$の各次元に対して二値分類（弱識別）を実行する関数です．パラメータ$z$によって符号を判別します．
* 仮説空間：$\mathcal{H}=\left\{h_{\alpha, k, z}(x) \mid \alpha \in\{-1,+1\}, k \in\{1, \ldots, d\}, z \in \mathbb{R}\right\}$

このとき，決定株の仮説空間のラデマッハ複雑度は

$$
R_n(\mathcal{H})=\sqrt{\frac{2}{n} \log (2(n+1) d)}
$$

で与えられます．

**証明**

これはMassartの有限仮説の補題で証明できます．復習すると，有限集合$\mathcal{A} \subset \mathbb{R}^n$に対して，$\sup _{\left(a_1, \ldots, a_n\right) \in \mathcal{A}} \sum_{i=1}^n a_i^2 \leq M^2$ならば，

$$
\mathbb{E}_\sigma\left[\sup _{\left(a_1, \ldots, a_n\right) \in \mathcal{A}} \frac{1}{n} \sum_{i=1}^n \sigma_i a_i\right] \leq \frac{\sqrt{2 M^2 \log | \mathcal{A}}}{n} |
$$
が成り立ちます．

ここで，決定株の前提を思い出しましょう．
* $\alpha \in\{-1,+1\}$ 及び $k \in\{1, \ldots, d\}$は離散集合です
* $z\in \mathbb{R}$は連続ですが，各座標$k=1, \dots, d$について，$x_1, x_2, \dots, x_d$を考えると，それぞれの値を超えるか超えないかの２種類しか仮説関数に反映されません．よって，仮説としての種類は実質$n+1$通りしかありません．
* よって$\mathcal{H}$は有限集合であり，$|\mathcal{H}|=2 \times d \times(n+1)$なので，Massartの有限仮説の補題から，
$$
R_n(\mathcal{H})=\sqrt{\frac{2}{n} \log |\mathcal{H}|}=\sqrt{\frac{2}{n} \log (2(n+1) d)}
$$
が成り立ちます．

## 回帰木の汎化誤差解析

回帰木の詳細はスキップします．
次を準備として考えます：

* 各葉$\mathrm{leaf}_l$は値$w_l \in \mathbb{R}$を持つ
* 入力$x \in \mathcal{X} \subset \mathbb{X}^d$を葉のいずれかに対応付けるノード分割関数を決定木を使って表現する．
* 入力 $x$ が葉 $\mathrm{leaf}_l$に対応付けられる場合は $1$，そうでない場合は $0$ であるような対応付けを $\mathbb{1}_{x \in \text {leaf }_l}$, と表記する.
* 回帰木$h: \mathbb{R}^d \to \mathbb{R}$は
$$
h(x) := \sum_{l \in \mathrm{leaves(h)}} w_l \mathbb{1}_{x \in \mathrm{leaf}_l}
$$
として定義される．ここで$\mathrm{leaves(h)}$は$h$の葉の集合．

つまり，入力された$x$から葉$l$にたどり着いたら$w_l$がアクティブになり，回帰として出力されることになります．
また，ここでのパラメータは葉と対応付け，そして$w_l$になります．
このラデマッハ複雑度を導出しましょう．

### 回帰木のラデマッハ複雑度

* $\widehat{\boldsymbol{\sigma}}$を次元数=葉の数のベクトルとして，その要素を
$$
[\widehat{\boldsymbol{\sigma}}]_l=\sum_{i=1}^n \sigma_i \mathbb{1}_{x_i \in \mathrm{leaf}_l}
$$
とします．
* $\|\boldsymbol{w}\|_q \leq \lambda, q \geq 1$と仮定しましょう（アルゴリズムに実装する際は正則化をかければ良いです．）
* $h(x)$を構成する決定木のノード数を$m$とします．
* $h(x)$の集合を$\mathcal{H}_{m, \lambda, q}$とします．ここで，
  
$$
\mathcal{H}_{m, \lambda, q}=\left\{h(x)=\sum_{l \in \mathrm{leaves (h)}} w_l \mathbb{1}_{x \in \mathrm{leaf}_l} : \|w\|_q \leq \lambda , q \geq 1\right \}
$$
です．
 
このとき，そのラデマッハ複雑度は

$$
\begin{aligned}
\widehat{R}_{\mathcal{S}_n}\left(\mathcal{H}_{m, \lambda, q}\right)&=\frac{1}{n} \mathbb{E}\left[\sup _{h \in \mathcal{H}_{m, \lambda, q}}\left[\sum_{i=1}^n \sigma_i h\left(x_i\right)\right]\right]\\
&=\frac{1}{n} \underset{E}{\mathbb{E}}\left[\sup _{h \in \mathcal{H}_{m, \lambda, q}^{\circ}}[\hat{\sigma} \cdot \boldsymbol{w}]\right]\\
& \leq \frac{1}{n} \underset{\sigma}{\mathbb{E}}\left[\sup _{h \in \mathcal{H}_{m, \lambda, q}}\|\hat{\sigma}\|_r\|\boldsymbol{w}\|_q\right] \quad \text { (Hölder's inequality) } \\
& \leq \frac{\lambda}{n} \underset{\sigma}{\mathbb{E}}\left[\sup _{h \in \mathcal{H}_m}\|\hat{\sigma}\|_r\right] \quad\left(\|\boldsymbol{w}\|_q \leq \lambda\right) \\
& \leq \frac{\lambda}{n} \underset{\sigma}{\mathbb{E}}\left[\sup _{h \in \mathcal{H}_m}\|\hat{\sigma}\|_1\right] \quad\left(r \geq 1 \Rightarrow\|\hat{\sigma}\|_r \leq\|\hat{\sigma}\|_1\right)
\end{aligned}
$$

逆に言うと，Holderの不等式などを使いたいので適切な仮定を導入したわけですね．
ここでさらに，

$$
\begin{aligned}
&=\frac{\lambda}{n} \underset{\sigma}{\mathbb{E}}\left[\sup _{h \in \mathcal{H}_m} \sum_{l \in \operatorname{leaves}(h)}\left|\sum_{i=1}^n \sigma_i \mathbb{1}_{\left\{x_i \in \text { leaf }_l\right\}}\right|\right]\\
&\leq\frac{\lambda}{n} \underset{\sigma}{\mathbb{E}}\left[\sup _{h \in \mathcal{H}_m, s_l \in \{+1, -1\}} \sum_{l \in \operatorname{leaves}(h)}s_l\sum_{i=1}^n \sigma_i \mathbb{1}_{\left\{x_i \in \text { leaf }_l\right\}}\right]\\
&=\frac{\lambda}{n} \underset{\sigma}{\mathbb{E}}\left[\sup _{h \in \mathcal{H}_m, s_l \in \{+1, -1\}} \sum_{i=1}^n \sigma_i \sum_{l \in \operatorname{leaves}(h)}s_l\mathbb{1}_{\left\{x_i \in \text { leaf }_l\right\}}\right]
\end{aligned}
$$
とできます．最終的にこれは決定木に帰着できました！そもそも$w_l$を使った回帰問題でのラデマッハ複雑度を考えるのが難しいので，決定木に直していると考えるとわかりやすいでしょう．
結局，まとめると，

決定木$b: \mathbb{R}^d \rightarrow\{+1,-1\}$
$$
b(x ; \boldsymbol{s}, h):=\sum_{l \in \mathrm{leaves}(h)} s_l \mathbb{1}_{x \in \mathrm{leaf}_l}
$$
の集合を$\mathcal{B}$とすると，

$$
\begin{aligned}
& \widehat{R}_{\mathcal{S}_n}\left(\mathcal{H}_{m, \lambda, q}\right) \leq \frac{\lambda}{n} \underset{\underset{\sigma}{\mathbb{E}}}{\mathbb{E}}\left[\sup _{\substack{h \in \mathcal{H}_m \\
s_l \in\{+1,-1\}}} \sum_{i=1}^n \sigma_i \sum_{l \in \text { leaves }(h)} s_l \mathbb{1}_{\left\{x_i \in \text { lea } A_l\right\}}\right] \\
& =\frac{\lambda}{n} \underset{\sigma}{\mathbb{E}}\left[\sup _{\substack{h \in \mathcal{H}_m \\
s_l \in\{+1,-1\}}} \sum_{i=1}^n \sigma_i b\left(x_i ; s, h\right)\right] . \\
&
\end{aligned}
$$

であり，つまり
$$
\widehat{R}_n\left(\mathcal{H}_{m, \lambda, q}\right) \leq \frac{\lambda}{n} \underset{\sigma}{\mathbb{E}}\left[\sup _{b \in \mathcal{B}_m} \sum_{i=1}^n \sigma_i b\left(x_i\right)\right]=\lambda \widehat{R}_{\mathcal{S}_n}(\mathcal{B})
$$
A
として，**回帰木の複雑度が決定木の複雑度で抑えられます**．
この決定木$b(\cdot)$のノード数を$m$の二分木として，以下で解析していきましょう．
このとき$\mathcal{B}_m$は有限集合なので，Massartの有限仮説の補題から

$$
\widehat{R}_n\left(\mathcal{B}_m\right)=\sqrt{\frac{2}{n} \log \left|\mathcal{B}_m\right|}
$$

が成り立ちます．あとは$|\mathcal{B}_m|$を求めるだけです．

### 二分木の仮説数

これは簡単です．深さが$D$のとき，葉の数は$2^D$であり，ノード数は$m = 2^D - 1$なので，結局葉の数は$m+1$個です．ゴニョゴニョすると，結局
$\mathcal{B}_{m, 0}$（これは分類のしきい値を$0$にしたやつです）のサイズは
$$
\left|\mathcal{B}_{m, 0}\right|=d^m \cdot 2^{m+1}
$$
で与えられます．$\theta$の値をノードごとに帰る場合は$\theta$が連続になりそうですが，上でやった議論と同様にすると結局有限個の分類パターンしかありません．よって，

$$
\left|\mathcal{B}_m\right|=(n+1)^m \cdot d^m \cdot 2^{m+1}
$$

が得られます．